In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pickle

filename = 'finalized_model.sav'
pickle.dump(ann, open(filename, 'wb'))

ann = pickle.load(open(filename, 'rb'))

In [ ]:
from sklearn import set_config
set_config(display="diagram")

'''

numeric_lst = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df = df.select_dtypes(include=numeric_lst)
numeric_col = list(df.select_dtypes(include=numeric_lst).columns) 

'''

"\nnumeric_lst = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']\ndf = df.select_dtypes(include=numeric_lst)\nnumeric_col = list(df.select_dtypes(include=numeric_lst).columns) \n\n"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Preprocessing Tools

## Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
df = pd.read_csv('/content/drive/MyDrive/Machine Learning A-Z (Codes and Datasets)/Part 1 - Data Preprocessing/Section 2 -------------------- Part 1 - Data Preprocessing --------------------/Data.csv')
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
'''
best_features = SelectKBest(score_func=f_classif)
best_features.fit(X, y)
bfdf = pd.DataFrame(data = best_features.scores_, columns=['score'])
bfdf['features'] = X.columns
bfdf.nlargest(8, 'score')
'''

"\nbest_features = SelectKBest(score_func=f_classif)\nbest_features.fit(X, y)\nbfdf = pd.DataFrame(data = best_features.scores_, columns=['score'])\nbfdf['features'] = X.columns\nbfdf.nlargest(8, 'score')\n"

## Importing the dataset

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Machine Learning A-Z (Codes and Datasets)/Part 1 - Data Preprocessing/Section 2 -------------------- Part 1 - Data Preprocessing --------------------/Data.csv')
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
dataset.head()

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


## Taking care of missing data

In [ ]:
from sklearn.impute import SimpleImputer
Imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X[:, 1:3] = Imputer.fit_transform(X[:, 1:3])

## Encoding categorical data

### Encoding the Independent Variable

In [ ]:
'''categorical_cols = df['Country'] 

X = pd.concat([pd.get_dummies(categorical_cols), df.drop(columns= ['Country', 'Purchased'])], axis=1).values'''

"categorical_cols = df['Country'] \n\nX = pd.concat([pd.get_dummies(categorical_cols), df.drop(columns= ['Country', 'Purchased'])], axis=1).values"

In [ ]:
'''
for _ in df.select_dtypes(object).columns:
    print(np.sort(df[_].unique()))
'''

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
X

array([[1.0, 0.0, 0.0, 44.0, 72000.0],
       [0.0, 0.0, 1.0, 27.0, 48000.0],
       [0.0, 1.0, 0.0, 30.0, 54000.0],
       [0.0, 0.0, 1.0, 38.0, 61000.0],
       [0.0, 1.0, 0.0, 40.0, 63777.77777777778],
       [1.0, 0.0, 0.0, 35.0, 58000.0],
       [0.0, 0.0, 1.0, 38.77777777777778, 52000.0],
       [1.0, 0.0, 0.0, 48.0, 79000.0],
       [0.0, 1.0, 0.0, 50.0, 83000.0],
       [1.0, 0.0, 0.0, 37.0, 67000.0]], dtype=object)

### Encoding the Dependent Variable

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

le.classes_

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train[:, 3:] = sc.fit_transform(X_train[:, 3:])
X_test[:, 3:] = sc.transform(X_test[:, 3:])

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
#classifier.predict(X_test)
classifier.score(X_test, y_test)

0.5

### Actual vs prediction of test dataset

In [ ]:
# 1 is for Malignant and 0 for Benign
ts = pd.DataFrame(data=y_test, columns=['actual result'])
ts['predicted result'] = classifier.predict(X_test)
ts

,actual result,predicted result
0,0,1
1,1,1


### Confusion matrix and accuracy

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix: ')
print(cm)
print('Accuracy: ', accuracy_score(y_test, y_pred)*100, '%')

Confusion matrix: 
[[0 1]
 [0 1]]
Accuracy:  50.0 %


### K-Fold cross validation score & standard deviation

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 3)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 50.00 %
Standard Deviation: 13.61 %


# All Linear Models

In [ ]:
'''
# Polynomial
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly_reg = PolynomialFeatures(degree = 4)
X_poly = poly_reg.fit_transform(df)
poly_regressor = LinearRegression()
poly_regressor.fit(X_poly, y)

# SVR
# Scaler >
from sklearn.svm import SVR
SVR(kernel = 'rbf')

# Decision Tree
from sklearn.tree import DecisionTreeRegressor
DecisionTreeRegressor(random_state = 0)

# Random Forest
from sklearn.ensemble import RandomForestRegressor
RandomForestRegressor(n_estimators = 10, random_state = 0)
'''

"\n# Polynomial\nfrom sklearn.preprocessing import PolynomialFeatures\nfrom sklearn.linear_model import LinearRegression\npoly_reg = PolynomialFeatures(degree = 4)\nX_poly = poly_reg.fit_transform(df)\npoly_regressor = LinearRegression()\npoly_regressor.fit(X_poly, y)\n\n# SVR\n# Scaler >\nfrom sklearn.svm import SVR\nSVR(kernel = 'rbf')\n\n# Decision Tree\nfrom sklearn.tree import DecisionTreeRegressor\nDecisionTreeRegressor(random_state = 0)\n\n# Random Forest\nfrom sklearn.ensemble import RandomForestRegressor\nRandomForestRegressor(n_estimators = 10, random_state = 0)\n"

#Classification all models

In [ ]:
'''
#SVC
# Scaler >
from sklearn.svm import SVC
SVC(kernel = 'linear', random_state = 0)

#Logistic
# scaler >
from sklearn.linear_model import LogisticRegression
LogisticRegression(random_state = 0)

# KNN
# Scaler >
from sklearn.neighbors import KNeighborsClassifier
KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)

# Naive Bayes
# Scaler >
from sklearn.naive_bayes import GaussianNB
GaussianNB()

# Decision Tree
# Scaler >
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier(criterion = 'entropy', random_state = 0)

# Random Forest
# Scaler
from sklearn.ensemble import RandomForestClassifier
RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
'''

"\n#SVC\n# Scaler >\nfrom sklearn.svm import SVC\nSVC(kernel = 'linear', random_state = 0)\n\n#Logistic\n# scaler >\nfrom sklearn.linear_model import LogisticRegression\nLogisticRegression(random_state = 0)\n\n# KNN\n# Scaler >\nfrom sklearn.neighbors import KNeighborsClassifier\nKNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)\n\n# Naive Bayes\n# Scaler >\nfrom sklearn.naive_bayes import GaussianNB\nGaussianNB()\n\n# Decision Tree\n# Scaler >\nfrom sklearn.tree import DecisionTreeClassifier\nDecisionTreeClassifier(criterion = 'entropy', random_state = 0)\n\n# Random Forest\n# Scaler\nfrom sklearn.ensemble import RandomForestClassifier\nRandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)\n"

In [ ]:
'''
from sklearn.model_selection import GridSearchCV
import warnings
warnings.catch_warnings()
warnings.simplefilter('ignore')

params = {'n_neighbors':[i for i in range(1,51)]}

clf1 = GridSearchCV(clf,params)
clf1.fit(X_train,y_train)
clf1.best_params_
'''

"\nfrom sklearn.model_selection import GridSearchCV\nimport warnings\nwarnings.catch_warnings()\nwarnings.simplefilter('ignore')\n\nparams = {'n_neighbors':[i for i in range(1,51)]}\n\nclf1 = GridSearchCV(clf,params)\nclf1.fit(X_train,y_train)\nclf1.best_params_\n"